# Stochastic Variational Inference for LDA

## Introducción

En el presente notebook se realiza una implementación del algoritmo para topic modelling, Latent Dirichlet Allocation (LDA), en su versión con Stochastic Variational Inference. La implementación trata de apegarse a lo propuesto por Hoffman, Blei y Wang en [Stochastic Variational Inference](http://www.columbia.edu/~jwp2128/Papers/HoffmanBleiWangPaisley2013.pdf).
Otros trabajos que fueron tomados como referencia son:
 * Para un panorama más amplio sobre LDA: [Inference Methods for Latent Dirichlet Allocation](http://times.cs.uiuc.edu/course/598f16/notes/lda-survey.pdf);
 * Para una explicación más detallada sobre la Mean-Field Variational Family: [Variational Inference: A Review for Statisticians](https://arxiv.org/pdf/1601.00670.pdf)
 
Adicionalmente, la idea de implementar LDA surgió de un intento previo de querer implementar la propuesta de Wang y Blei, en [Collaborative Topic Modeling for Recommending Scientific Articles](http://www.cs.columbia.edu/~blei/papers/WangBlei2011.pdf), para Sistemas de Recomendación. De hecho el dataset utilizado aqui es el mismo que usan ellos. No descarto como siguiente paso intentar un Content-Based Recommender System.

## Marco teórico

### Modelo Generativo

Asumiendo K tópicos, y D documentos, cada uno con N palabras (por simplicidad asumo documentos de igual longitud) pertenecientes a un vocabulario de tamaño V; el modelo generativo es el siguiente:

1. Generamos tópicos $\beta_k$ ~ Dirichlet($\eta,\dots,\eta$) para cada $k\in \{0,\dots,K-1\}$
2. Para cada documento $d \in \{0,\dots, D-1\}$:
    3. Generamos $\theta_d$ ~ Dirichlet($\alpha,\dots,\alpha$)
    4. Para cada palabra $w \in \{0,\dots, N-1\}$:
        5. Generamos la asignación de tema $z_{dn}$ ~ Multinomial($\theta_d$)
        6. Generamos la palabra $w_{dn}$ ~ Multinomial($\beta_{z_{dn}}$)

Los parámetros involucrados son:
* $\beta_k$, el tópico $k$. Un tópico consiste básicamente en un vector de probabilidad de longitud V, que modela la distribución de las distintas palabras en el vocabulario para ese tema
* $\theta_d$, la proporción de los tópicos para el documento $d$. Nuevamente es un vector de probabilidades, que en este caso apunta a modelar la participación parcial de los distintos temas para un mismo documento. El hecho de que se permita que un documento pertenezca a un mixture de tópicos en lugar de uno solo, es una de las principales virtudes de LDA. __Esta es la principal variable sobre la que nos interesa poder hacer inferencia__.
* $z_{dn}$, es la asignación del tópico de la palabra $n$ en el domunento $d$. Por cuestiones prácticas es un one-hot vector K-dimensional
* $w_{dn}$, es la palabra $n$ del documento $d$. Pertenece al tópico $z_{dn}$. Por cuestiones prácticas es un one-hot vector V-dimensional

![title](img/lda-graphical-model.png)

### Variational Distribution

Para estimar las distribuciones de arriba se utilizan las siguientes familias variacionales:
* Para las asignaciones de tópicos, usamos q($z_{dn}$) ~ Multinomial($\phi_{dn}$)
* Para las proporciones de tópicos en cada documentos, q($\theta_{d}$) ~ Dirichlet($\gamma_{d}$)
* Para las distribuciones de probabilidad sobre las palabras de cada tópico, q($\beta_{k}$) ~ Dirichlet($\lambda_{k}$)

En particular, $\gamma$ y $\lambda$ son los parámetros de mayor interés como output del modelo, ya que el primero es el que nos permite tener la distribución de tópicos para un documento visto, mientras que el segundo nos permite hacer la inferencia de estas proporciones sobre nuevos documentos.

## Dataset

El dataset fue descargado de http://www.cs.cmu.edu/~chongw/data/citeulike/. El mismo contiene información del sitio http://www.citeulike.org/, que le permite a investigadores armar sus bibliotecas de papers y recibir recomendaciones en base a las mismas. En particular yo no uso la información referente a los usuarios y sus bibliotecas, si no que únicamente me centro en la data sobre artículos.
Los dos archivos que uso son:
* mult.dat, que contiene los ids de las palabras más relevantes de cada artículos, y sus conteos. En total son 16980 artículos (documentos).
* vocab.dat, el mapeo de los ids a la palabra específica. El tamaño del vocabulario usado es de 8000 palabras.

La extracción del vocabulario, y demás preprocesamientos de la data son los que se explican en la sección 4 de [Wang and Blei (2011)](http://www.cs.columbia.edu/~blei/papers/WangBlei2011.pdf).

Adicionalmente, en esta primera iteración del algoritmo, pido que todos los documentos tengan la misma cantidad de palabras. Eso me lleva a realizar un procesamiento extra, en el cual al cargar los documentos hago un resampling (con reposición) para tener 60 palabras por documento (esto es totalmente arbitrario).

In [1]:
from scipy.stats import dirichlet, multinomial, expon
from scipy.special import digamma
from collections import defaultdict
from math import pow
import numpy as np
import copy
import logging

from online_lda import OnlineLDA
from mini_batch_lda import MiniBatchLDA
from helpers import *

logging.basicConfig(level=logging.INFO)

In [2]:
def load_dataset():
    words_per_doc = defaultdict(list)
    # Parse line by line the documents data's file
    with open('data/mult.dat') as keywords_f:
        words_and_counts_by_doc = []
        for l in keywords_f:
            data = l.split() # First element is the number of distinct words, the rest are the word_ids with its counters 
            words_and_counts_by_doc.append(data[1:])
        for d, words_and_counts in enumerate(words_and_counts_by_doc):
            for w_c in words_and_counts:
                word, cnt = w_c.split(':')
                for _ in xrange(int(cnt)):    
                    words_per_doc[d].append(int(word))
            # Resampling the data to always have the same number of words per document
            np.random.seed(42)
            words_per_doc[d] = np.random.choice(words_per_doc[d], size=60)
    
    vocab = {}
    with open('data/vocab.dat') as vocab_f:
        for v, word in enumerate(vocab_f):
            vocab[v] = word.rstrip()
    
    return words_per_doc, vocab
            

In [ ]:
words_by_doc, vocab = load_dataset()

## LDA

Primero me defino algunas funciones auxiliares que me van a servir luego.

La clase _LatentDirichletAllocation_ tiene como método principal _fit_, que es el que se encarga de aplicar el algoritmo propuesto por Hoffman et al. (2013). La notación elegida sigue fielmente a la usada en el paper.

La imagen siguiente muestra el pseudocódigo del algoritmo en cuestión.

![title](img/SVI-pseudocode.png)

### Minibatch LDA

In [ ]:
D = len(words_by_doc)  # number of documents
N = 60 # words per doc
K = 30  # number of topics
V = len(vocab)  # vocabulary size
minibatch_size = 128

# For alpha and eta values I follow suggestion from Griffiths and Steyvers (2004)
lda = MiniBatchLDA(K, D, N, V, 50.0/K, 0.1, words_by_doc, minibatch_size)
lda.fit()

INFO:mini_batch_lda:Iteration: 0
INFO:mini_batch_lda:Ro: 1.0
INFO:mini_batch_lda:Iteration: 1
INFO:mini_batch_lda:Ro: 0.574349177499
INFO:mini_batch_lda:Iteration: 2
INFO:mini_batch_lda:Ro: 0.415243646539
INFO:mini_batch_lda:Iteration: 3
INFO:mini_batch_lda:Ro: 0.329876977693
INFO:mini_batch_lda:Iteration: 4
INFO:mini_batch_lda:Ro: 0.275945932292
INFO:mini_batch_lda:Iteration: 5
INFO:mini_batch_lda:Ro: 0.238494846851
INFO:mini_batch_lda:Iteration: 6
INFO:mini_batch_lda:Ro: 0.210824737371
INFO:mini_batch_lda:Iteration: 7
INFO:mini_batch_lda:Ro: 0.189464570814
INFO:mini_batch_lda:Iteration: 8
INFO:mini_batch_lda:Ro: 0.172427285991
INFO:mini_batch_lda:Iteration: 9
INFO:mini_batch_lda:Ro: 0.158489319246
INFO:mini_batch_lda:Iteration: 10
INFO:mini_batch_lda:Ro: 0.1468540242
INFO:mini_batch_lda:Iteration: 11
INFO:mini_batch_lda:Ro: 0.136979319126
INFO:mini_batch_lda:Iteration: 12
INFO:mini_batch_lda:Ro: 0.128482896333
INFO:mini_batch_lda:Iteration: 13
INFO:mini_batch_lda:Ro: 0.121087014505
I

In [ ]:
for i, lamb_k in enumerate(lda.lamb):
    print_md('Topic {}'.format(i), bold=True)
    np.random.seed(42)
    words_distribution = np.random.dirichlet(lamb_k)
    #print words_distribution.argsort()[::-1][:10]
    #words_distribution.sort()
    #print words_distribution[::-1][:10]
    print_words(words_distribution.argsort()[::-1][:20], vocab)

In [ ]:
print_top_k_topics_for_doc(lda, 2000, 10, words_by_doc, vocab)

## Online LDA

In [ ]:
D = len(words_by_doc)  # number of documents
N = 60 # words per doc
K = 60  # number of topics
V = len(vocab)  # vocabulary size

# For alpha and eta values I follow recommendation from Griffiths and Steyvers (2004)
lda = OnlineLDA(K, D, N, V, 50.0/K, 0.1, words_by_doc)
lda.fit()

In [ ]:
for i, lamb_k in enumerate(lda.lamb):
    print_md('Topic {}'.format(i), bold=True)
    np.random.seed(42)
    words_distribution = np.random.dirichlet(lamb_k)
    #print words_distribution.argsort()[::-1][:10]
    #words_distribution.sort()
    #print words_distribution[::-1][:10]
    print_words(words_distribution.argsort()[::-1][:20], vocab)